In [101]:
import os
import requests
import pandas as pd
import io
import re
import json
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import imgkit
import openpyxl as px
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage


def parse_domain(url):
    domain = urlparse(url).netloc.replace('www.', '')
    return domain

def request_csv(url):
    response = requests.get(url)
    response.raise_for_status()
    return pd.read_csv(io.StringIO(response.text)).to_dict(orient='records')

def get_CISA_MAP():
    if os.path.exists('CISA_MAP.json'):
        with open('CISA_MAP.json', 'r') as f:
            CISA_MAP = json.load(f)
    else:
        CISA_MAP = {}
        CISA = request_csv('https://github.com/cisagov/dotgov-data/blob/main/current-federal.csv?raw=true')
        for row in CISA:
            CISA_MAP[row['Domain name']] = row
        with open('CISA_MAP.json', 'w') as f:
            json.dump(CISA_MAP, f)
    return CISA_MAP

def cut_text_to_n_tokens(text, n=500):
    tokens = text.split()
    if len(tokens) > n:
        text = ' '.join(tokens[:n])
    return text

def get_soup(url):
    response = requests.get(url)
    response.raise_for_status()
    return BeautifulSoup(response.text, 'html.parser')

def short_soup(soup):
    for tag in soup.find_all(['script', 'style']):
        tag.decompose()
    return soup

def screenshot(url):
    image_location = f'{url.split("/")[-1]}_screenshot.jpg'
    imgkit.from_url(url, image_location)
    return image_location


def whosAgency(url, temperature=0.2):
    CISA_MAP = get_CISA_MAP()
    domain = parse_domain(url)
    if domain in CISA_MAP:
        agency_response = CISA_MAP[domain]
    else:
        soup = get_soup(url)
        links = [{x['href']: x.text} for x in soup.find_all('a') if x.get('href') and x.get('href').startswith('http')]
        text = soup.get_text()
        link_json = json.dumps(links)
        token_cut = 1500
        text = cut_text_to_n_tokens(text, token_cut - len(link_json.split()))
        website_info = {url: {'text': text, 'links': links}}

        messages = [
            SystemMessage(content="""You are an AI assistant trained to analyze website content and determine the US federal agency that owns or is responsible for the website. You should focus on textual content, official logos, disclaimers, contact information, and any relevant metadata. All responses must be in this json format:  
                            {
                                'url': {dhs sub-bureau non-dot-gov url},
                                'Domain name': 'dhs.gov',
                                'Domain type': 'Federal - Executive',
                                'Agency': 'Department of Homeland Security',
                                'Organization name': 'Headquarters',
                                'City': 'Washington',
                                'State': 'DC',
                                'Security contact email': 'IS2OSecurity@hq.dhs.gov'
                            }
                                we want to union this information with the CISA MAP. If you can't determine the agency, please don't guess. simply fill in the field as "NONE FOUND" in the JSON response."""),
            HumanMessage(content=f"Please fill out the metadata json if you can determine the agency that is responsible for {website_info}? Please fill out the requested metadata json for this url. Also - make sure to use formal names, never abbreviations, nicknames, or monikers.")
        ]
        
        llm = ChatOpenAI(api_key=api_key, model="gpt-3.5-turbo", temperature=temperature)
        agency_response = llm(messages)
    messages = [
        SystemMessage(content="""You are an AI assistant trained to analyze website content and determine which US Program Office collected or published the data. You should focus on textual content, official logos, disclaimers, contact information, and any relevant metadata. All responses must be in this json format:  
                        {
                            'url': 'https://github.com/cisagov',
                            'program name': 'Cybersecurity and Infrastructure Security Agency (CISA)',
                            'program main url': 'https://www.cisa.gov/',
                            'Domain name': 'dhs.gov',
                            'Domain type': 'Federal - Executive',
                            'Agency': 'Department of Homeland Security',
                            'Organization name': 'Headquarters',
                            'City': 'Washington',
                            'State': 'DC',
                            'Security contact email': 'IS2OSecurity@hq.dhs.gov'
                        }
                            we want to union this information with the Federal Budget Database. If you can't determine the specific program office, look for the most prominent text on the page taht loooks like a title and use a derivation of that with a (prelim) note. please don't guess without providing evidence that the results are imperfect"."""),
        HumanMessage(content=f"Please fill out the metadata json if you can determine the program office that is responsible for {website_info}? Please fill out the requested metadata json for this url. Also - make sure to use formal names, never abbreviations, nicknames, or monikers.")
    ]
    
    llm = ChatOpenAI(api_key=api_key, model="gpt-3.5-turbo", temperature=temperature)
    source_response = llm(messages)
    
    return agency_response, source_response

def auto_sourcery(url):
    agency_response, source_response = whosAgency(url, temperature=0.5)
    agency = json.loads(agency_response.content.replace("'", '"'))
    source = {} if source_response is None else json.loads(source_response.content.replace("'", '"'))

    agency['agency_display_name'] = agency['Agency']
    agency['source_display_name'] = source.get('program name', '')
    agency['source_display_url'] = source.get('program main url', '')
    agency['agency_display_url'] = f'https://www.{agency["Domain name"]}'
    return agency

def get_data_to_map(data_path):
    data = px.load_workbook(data_path, read_only=True)
    data_dict = {}
    for sheet in data.sheetnames:
        data_dict[sheet] = pd.DataFrame(data[sheet].values)
    return data_dict

def get_data_path_map(data_path):
    # fact_results = []
    df = get_data_to_map(data_path)['result']
    df.columns = df.iloc[0]
    df = df.drop(0)
    return df

def categorize_column(df,series):
    # check to see if it has time. 
    if any([x in series.lower() for x in time_headers]):
        return 'TIME__'
    # check to see if it has location
    if any([x in series.lower() for x in location_headers]):
        return 'LOC__'
    # check to see if it has value
    if any([x in series.lower() for x in standard_facts]):
        return 'FACT__'
    if any([x in series.lower() for x in standard_dims]):
        return 'DIM__'
    if len(df[series]) < (len(df[series].unique()) * 3):
        return 'FACT__'
    return 'DIM__'


def publish_dimension(prefix, series_name, unique_value):
    if unique_value:
        return {'raw_column': series_name, 'transformed_column': f'{prefix}{series_name}'} #, 'raw_value': unique_value, 'transformed_value': unique_value}
    return {'raw_column': series_name, 'transformed_column': f'{prefix}{series_name}'}

def publish_fact(series):
    return {'fact_column_name': series.name, 'fact_display_name': f'{prefix}{replace_special_chacters_and_spaces(series.name)}'}
def publish_table(table_lookups):
    table_lookups = {k: v for k, v in table_lookups.items() if k in ['source_display_name', 'source_display_url', 'agency_display_name', 'agency_display_url']}
    table_lookups = pd.DataFrame(table_lookups, index=[0]).drop_duplicates()
    return table_lookups
def replace_special_chacters_and_spaces(string):
    return re.sub(r'\W+', '_', string)

def dfs_to_excel(dfs_dict, data_path):
    """
    Converts a dictionary of pandas DataFrames to an Excel file with multiple sheets.

    Parameters:
    dfs_dict (dict): A dictionary where the keys are sheet names and the values are DataFrames.
    file_path (str): The path to save the Excel file.
    """
    with pd.ExcelWriter(data_path, engine='xlsxwriter') as writer:
        for sheet_name, df in dfs_dict.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
        writer.close()  # Add this line to save the Excel file

        
# Define patterns for identifying date/year columns and location columns
date_patterns = [r'^\d{4}$', r'\d{4}-\d{2}-\d{2}', r'\d{2}/\d{2}/\d{4}', r'\d{2}-\d{2}-\d{4}']
location_keywords = ['state', 'united states', 'county', 'metro', 'city', 'town', 'location', 'region', 'geographic']
number_pattern = re.compile(r'^\d+(\.\d+)?$')

def what_is_being_measured(context, url, temperature=0.2, soup=None):
    if not soup:
        soup = get_soup(url)
    text = soup.get_text()
    website_info = cut_text_to_n_tokens(text, 1000)

    messages = [
        SystemMessage(content="""You are an AI assistant trained to analyze website content and determine what is being measured, the unit of measurement, the field type, and a easily understood display name.
Let me give you some examples: 
1. "The number of people in the United States is 330 million", you would respond with: {"fact_display_name": "number of people", "units": "people", "unit_display_name": "people", "unit_type": "INT"},
2. "'Department of State\nBureau of Population, Refugees, and Migration\nOffice of Admissions - Refugee Processing Center\nSummary of Refugee Admissions. as of 30-April-2024. Cumulative Summary of Refugee Admissions', you would respond with {"fact_display_name": "Refugee Admissions", "units": "people", "unit_display_name": "people", "unit_type": "INT"}.

Please respond in this JSON format:
{
    "fact_display_name": "measurement",
    "units": "unit of measure",
    "unit_display_name": "unit to display", # typically the same as unit
    "unit_type": "unit_type" # may be INT, FLOAT, or STRING
}"""),
        HumanMessage(content=f"Please review this information from the data file: {context} and this website information: {website_info}. Please fill out the requested metadata JSON for this URL.")
    ]
    
    llm = ChatOpenAI(api_key=api_key, model="gpt-3.5-turbo", temperature=temperature)
    fact_response = llm(messages)
    return json.loads(fact_response.content) if json.loads(fact_response.content) else None


In [94]:
OPENAI_API_KEY = 'sk-proj-pGa1oIafi3C89siUTdluT3BlbkFJbYpCeKCwrnLomY2zF96M'
url = 'https://www.wrapsnet.org/admissions-and-arrivals/'
data_path = '/Users/robertstillwell/robcode/mapping_work/Beta/RPC_Arrivals_Admissions.xlsx'
exclude_date_keywords = ['authorized', 'source']
forced_dimension_columns = ['table', 'source','header']
forced_fact_columns = ['Value']

url_soup = get_soup(url)
api_key = os.getenv('OPENAI_API_KEY') if os.getenv('OPENAI_API_KEY') else OPENAI_API_KEY
table_lookups = auto_sourcery(url)
mapped_data = []
time_headers = ['date', 'time', 'year', 'month', 'week', 'day','quarter']
location_headers = ['location', 'city', 'state', 'country', 'metro', 'country','nation','region','division']
standard_dims = []
standard_facts = ['Value']
dataset_mapping = []
fact_lookups = []
data_df = get_data_path_map(data_path)
ignore_location = True

for col in data_df.columns:
    series = data_df[col]
    prefix = categorize_column(data_df, col)
    if prefix == 'LOC__' and ignore_location:
        prefix = 'DIM__'
    if prefix in ('DIM__', 'LOC__', 'TIME'):
        uniques = series.unique()
        for unique in uniques:
            if pd.isna(unique) or unique.strip() == '':
                continue
            dimension = publish_dimension(prefix, series.name, unique)
            dataset_mapping.append(dimension)
    elif prefix == 'FACT__':
        context = data_df['Header'][1]
        fact_metadata = what_is_being_measured(context, url, temparature=0.2, soup=url_soup)
        fact_metadata['fact_column_name'] = series.name
        fact_metadata['fact_display_name'] = f"{prefix}{replace_special_chacters_and_spaces(fact_metadata['fact_display_name'])}"
        fact_metadata['transformed_column']: f'{prefix}{series.name}'
        fact_lookups.append(fact_metadata)
        dim_metadata = {'raw_column': series.name, 'transformed_column': fact_metadata['fact_display_name']}
        dataset_mapping.append(dim_metadata)

        # dataset_mapping.append([fact_metadata])
table_lookups = pd.DataFrame(table_lookups, index=[0])[['agency_display_name','agency_display_url','source_display_name','source_display_url']]

fact_lookups = pd.DataFrame(fact_lookups)
fact_lookups_columns = [x.strip() for x in 'fact_column_name	fact_display_name	unit	unit_display_name	unit_type	adjustments	adjustments_display_name	glossary_text'.split('\t')]
for column in fact_lookups_columns:
    if column not in fact_lookups.columns:
        fact_lookups[column] = None
        


dataset_mapping = pd.DataFrame(dataset_mapping)
dataset_mapping_columns = [x.strip() for x in 'acquisition_url	start_date	end_date	raw_column	raw_value	transformed_column	transformed_value	column_display_name	value_display_name	gold_table_name'.split('\t')]
for column in dataset_mapping_columns:
    if column not in dataset_mapping.columns:
        if column == 'acquisition_url':
            dataset_mapping[column] = data_path.split('/')[-1]
        else: 
            dataset_mapping[column] = None
dataset_mapping = dataset_mapping[dataset_mapping_columns].drop_duplicates()
results_dict = {'dataset_mapping':dataset_mapping, 'table_lookups':table_lookups, 'fact_lookups':fact_lookups}

dfs_to_excel(results_dict, data_path.replace('.xlsx','_mapping.xlsx'))
